## Data engineering

Build an entire pipe-line combining 2 different sources and output to Power BI service. For this, you will choose 2 tables. Table A from INE and table B from UE. You will later combine in the common field, preferably, `date`.

__1. Extract data from table A directly from INE using python__

You can use the notebook "State_Stats"

In [3]:
import pandas as pd
import requests

url_ine = 'https://servicios.ine.es/wstempus/js/ES/DATOS_TABLA/12157?date=' #PIB pm Oferta (Precios Corrientes) - Trimestrales

r = requests.get(url_ine)
data_ine = r.json()
original_data_ine = pd.DataFrame(data_ine)
selected_data_ine = original_data_ine.loc[original_data_ine['COD'] == 'EOT14624', ['COD', 'Data']]

aggregated_data_ine = {key: [] for key in selected_data_ine.iloc[0]['Data'][0].keys()}

for row in selected_data_ine['Data']:
    for dictionary in row:
        for key, value in dictionary.items():
            if key in aggregated_data_ine:
                if key == 'Fecha':  # Convert timestamp to desired date format
                    value = pd.to_datetime(value, unit='ms').strftime('%d-%m-%Y')
                aggregated_data_ine[key].append(value)

# Create a DataFrame from the aggregated data
table_a = pd.DataFrame(aggregated_data_ine)

# Group by year
average_per_year = table_a.groupby('Anyo')['Valor'].mean().reset_index()

# Rename the 'Anyo' column to 'year'
average_per_year.rename(columns={'Anyo': 'year'}, inplace=True)

# Rename 'Valor' for "Precios corrientes"
average_per_year.rename(columns={'Valor': 'Precios Corrientes'}, inplace=True)

# Convert the 'year' column to numeric and coerce errors to NaN
average_per_year['year'] = pd.to_numeric(average_per_year['year'], errors='coerce')

# Display the DataFrame after renaming and conversion
print(average_per_year)



    year  Precios Corrientes
0   2002           89.882500
1   2003           92.268333
2   2004           93.828333
3   2005           95.058333
4   2006           97.060833
5   2007           99.322500
6   2008          100.000000
7   2009           94.098333
8   2010           92.101667
9   2011           92.042500
10  2012           91.656667
11  2013           91.168333
12  2014           92.420000
13  2015           96.536667
14  2016          101.680000
15  2017          108.022500
16  2018          109.928333
17  2019          112.192500
18  2020          103.074545
19  2021          109.683333
20  2022          128.173333
21  2023          139.421667


__2. Extract data from table B directly from UE using python__

In [9]:
import eurostat
import pandas as pd

toc_df = eurostat.get_toc_df()
print(toc_df)

#Give the code to the function to receive the data
data = eurostat.get_data_df('POST_CUBE1_X$NUM701')
data

# Assign it to a df name
df = data

# Use melt to reshape the DataFrame
melted_df = pd.melt(df, id_vars=['FREQ', 'UNIT', 'INDIC_PS', 'GEO\TIME_PERIOD'], var_name='year', value_name='value')

# Display the reshaped DataFrame
melted_df

#Call only the columns that are of interest
df_final = melted_df[['year', 'value']]

#Convert year to numeric
df_final['year'] = pd.to_numeric(df_final['year'])
df_final

# Group by year
avg_per_year = df_final.groupby('year')['value'].mean().reset_index()

#Rename value for the name of the indicator
avg_per_year.rename(columns={'value': 'Number of companies that offer postal service'}, inplace=True)

avg_per_year


                                                  title  \
0     Country level - passengers embarked and disemb...   
1     Passengers embarked and disembarked in the top...   
2     Passengers (excluding cruise passengers) trans...   
3     Passengers (excluding cruise passengers) trans...   
4     Country level - number and gross tonnage of ve...   
...                                                 ...   
7727  Percentage of letters delivered on-time (USP u...   
7728                                    Postal services   
7729    Number of enterprises providing postal services   
7730  Access points  (USP under direct or indirect d...   
7731  Domestic postal traffic, letter mail and parce...   

                         code     type       last update of data  \
0              MAR_MP_AA_CPHD  dataset  2023-12-13T11:00:00+0100   
1              MAR_MP_AA_PPHD  dataset  2023-12-13T11:00:00+0100   
2               MAR_MP_AM_CFT  dataset  2023-12-13T11:00:00+0100   
3              MAR_

/var/folders/kx/ttlm9tqs63z73zxt8n3pptvm0000gn/T/ipykernel_55460/1821701417.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['year'] = pd.to_numeric(df_final['year'])


,year,Number of companies that offer postal service
0,2012,282.393939
1,2013,233.170455
2,2014,229.044944
3,2015,234.333333
4,2016,241.224719
5,2017,240.685393
6,2018,214.055556
7,2019,228.767442
8,2020,237.771084
9,2021,271.440000


__3. Join both tables using the common field__

You can use `merge` or `concat`

In [10]:
merged_df = pd.merge(average_per_year, avg_per_year, on='year', how='inner')

# Replace NaN values with 0
merged_df.fillna(0, inplace=True)

# Change year to a string
merged_df['year'] = pd.to_datetime(merged_df['year'], format='%Y')

merged_df


,year,Precios Corrientes,Number of companies that offer postal service
0,2012-01-01,91.656667,282.393939
1,2013-01-01,91.168333,233.170455
2,2014-01-01,92.420000,229.044944
3,2015-01-01,96.536667,234.333333
4,2016-01-01,101.680000,241.224719
5,2017-01-01,108.022500,240.685393
6,2018-01-01,109.928333,214.055556
7,2019-01-01,112.192500,228.767442
8,2020-01-01,103.074545,237.771084
9,2021-01-01,109.683333,271.440000


__4. Create a calculated field named "pred_place_holder"__

You can create a dummy column of `ones`. This step is to ilustrate "the magic goes here"

In [15]:
merged_df['pred_place_holder'] = 1
merged_df


,year,Precios Corrientes,Number of companies that offer postal service,pred_place_holder
0,2012-01-01,91.656667,282.393939,1
1,2013-01-01,91.168333,233.170455,1
2,2014-01-01,92.420000,229.044944,1
3,2015-01-01,96.536667,234.333333,1
4,2016-01-01,101.680000,241.224719,1
5,2017-01-01,108.022500,240.685393,1
6,2018-01-01,109.928333,214.055556,1
7,2019-01-01,112.192500,228.767442,1
8,2020-01-01,103.074545,237.771084,1
9,2021-01-01,109.683333,271.440000,1


__5. Push the resulting df to Power BI service__

You can use the notebook "Idealista"

In [16]:
import json
import requests

# Convert DataFrame to JSON
json_data = merged_df.to_json(orient='records')

# Convert JSON string to bytes
data_bytes = json_data.encode('utf-8')

url = 'https://api.powerbi.com/beta/78954451-ce2d-4c90-ae61-68906e409956/datasets/90ab2b0d-7eb1-498d-aca2-c278e02f279f/rows?redirectedFromSignup=1%2C1&ScenarioId=Signup%3Fpbi_source%3Dwebsignup_getstarted_hero&responseError=UserNotLicensed&ru=https%3A%2F%2Fapp.powerbi.com%2F%3Fpbi_source%3Dwebsignup_getstarted_hero%26redirectedFromSignup%3D1%26noSignUpCheck%3D1%26ScenarioId%3DSignup&redirectedWaitSimple=1&experience=power-bi&key=WkJbkN2rJqZw%2FjmeZrWgZijxV%2BBCHq6sc7Hbe8n5pJAu2Oyf3t4XHIWLzc%2BZy4OXylwXpkqaI0RADX05c4N9UA%3D%3D'
headers = {'Content-Type': 'application/json'}

# Send the POST request
response = requests.post(url, headers=headers, data=data_bytes)

# Print the response
response

<Response [200]>

__6. Paste the code you would use to automate the process using your local scheduler__

You can use the notebook "Local_deployment". You need 2 files: .py, and .bat.

In [ ]:
# deploy_script.py

import subprocess
import os

# Activate the virtual environment
subprocess.run(['/Users/andreguichard/Desktop/Master In Big Data/Revision de Bases de Datos/Entregable 2/env/bin/activate'])

# Change the current directory
os.chdir('/Users/andreguichard/Desktop/Master In Big Data/Revision de Bases de Datos/Entregable 2/')

# Run the Python script
subprocess.run(['python', 'app.py'])

# deploy_script.sh
source /Users/andreguichard/Desktop/Master\ In\ Big\ Data/Revision\ de\ Bases\ de\ Datos/Entregable\ 2/env/bin/activate
cd /Users/andreguichard/Desktop/Master\ In\ Big\ Data/Revision\ de\ Bases\ de\ Datos/Entregable\ 2/
python app.py

#Speciic time to run the code
0 2 * * * /path/to/python /path/to/deploy_script.py